In [8]:
# import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

In [11]:
# create path and set up browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [12]:
# scrape the latest news from the NASA Mars site
nasa_url = 'https://mars.nasa.gov/news'
browser.visit(nasa_url)

nasa_html = browser.html
nasa_soup = BeautifulSoup(nasa_html, 'html.parser')
title_results = nasa_soup.find_all('div', class_='content_title')
paragraph_results = nasa_soup.find_all('div', class_='article_teaser_body')
nasa_title = title_results[0].text
nasa_paragraph = paragraph_results[0].text

In [4]:
# grab the featured image from JPL website
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
browser.click_link_by_partial_text('FULL IMAGE')
browser.is_element_not_present_by_text('more info', wait_time=5)
browser.click_link_by_partial_text('more info')
jpl_html = browser.html
jpl_soup = BeautifulSoup(jpl_html, 'html.parser')
featured_image_url= 'https:' + jpl_soup.find_all('div', class_='download_tiff')[1].a['href']

In [5]:
# scrape the latest Mars weather tweet
twitter_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(twitter_url)
twitter_html = browser.html
twitter_soup = BeautifulSoup(twitter_html, 'html.parser')
mars_weather_tweets = twitter_soup.find_all('div', class_= 'js-tweet-text-container')

if mars_weather_tweets[0].p.text[:3] != '^Sol':
    mars_weather = mars_weather_tweets[1].p.text
else:
    mars_weather = mars_weather_tweets[0].p.text

In [6]:
# use pandas to scrape tabular data from Mars Facts
facts_url = 'https://space-facts.com/mars/'
mars_table = pd.read_html(facts_url)
mars_table
mars_df = mars_table[0]
mars_df.columns = ['Description', 'Value']
mars_df.set_index('Description', inplace=True)
mars_html_table = mars_df.to_html()

In [7]:
# scrape the mars hemispher images
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
usgs_response = requests.get(usgs_url)
usgs_soup = BeautifulSoup(usgs_response.text, 'html.parser')
pages = usgs_soup.find_all('a', class_="itemLink product-item")

urls = []
for page in pages:
    partial_url = page['href']
    full_url = "https://astrogeology.usgs.gov" + partial_url
    urls.append(full_url)

hemisphere_image_urls = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    image = soup.find('img', class_='wide-image')['src']
    image_url = 'https://astrogeology.usgs.gov' + image
    title = soup.find('h2', class_='title').text
    
    mars_dict = {"title": title,
                "img_url": image_url}
    
    hemisphere_image_urls.append(mars_dict)
    


print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
